### Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo


Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos a seguir son:
 
1. Preparación de Datos: Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.
    
2. Análisis Descriptivo: Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estándar, los percenEles...) del número de vuelos reservados para cada grupo.

3. Prueba Estadística: Realiza una prueba A/B Testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.


In [54]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

1. Preparación de datos

In [8]:
#cargamos csv y vemos las primeras filas:
df= pd.read_csv('customer_dataset.csv')
df.head()

,Loyalty Number,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,August,2017,January,3,0,3,1521,152.0,0,0
1,100102,Ontario,Toronto,M1R 4K3,Male,College,73479.0,Single,Nova,2887.74,Standard,2013,March,2017,January,10,4,14,2030,203.0,0,0
2,100140,British Columbia,Dawson Creek,U5I 4F1,Female,College,73479.0,Divorced,Nova,2838.07,Standard,2016,July,2017,January,6,0,6,1200,120.0,0,0
3,100214,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,August,2017,January,0,0,0,0,0.0,0,0
4,100272,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,January,2017,January,0,0,0,0,0.0,0,0


In [15]:
#sacamos las columnas que nos piden para el test A/B:
df_test = df[['Flights Booked', 'Education']]
df_test.head()

,Flights Booked,Education
0,3,Bachelor
1,10,College
2,6,College
3,0,Bachelor
4,0,Bachelor


2. Análisis descriptivo

In [21]:
#agrupamos por nivel educativo.
#calculamos las estadísticas descriptivas.
agrupacion_niveles = df_test.groupby("Education")["Flights Booked"]
estadisticas_descriptivas = agrupacion_niveles.describe().T
estadisticas_descriptivas

Education,Bachelor,College,Doctor,High School or Below,Master
count,252567.000000,102260.000000,17731.000000,18915.000000,12287.000000
mean,4.110288,4.169744,4.175512,4.176209,4.200700
std,5.221671,5.246040,5.256971,5.239267,5.213956
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,8.000000,8.000000,8.000000,8.000000,8.000000
max,21.000000,21.000000,21.000000,21.000000,21.000000


Explicación análisis descriptivo:

 - mean: Todos los niveles educativos presentan una media cercana a 4 vuelos reservados, lo que indica que, en promedio, los clientes reservan aproximadamente el mismo número de vuelos, independientemente de su nivel educativo.
 
 - std: La desviación estándar la encontramos en todos los casos cerca de 5, lo que indica una alta variabilidad en el número de vuelos reservados.
 
 - mediana (50%): La mediana de 1 vuelo en todos los niveles sugiere que la mayoría de los clientes reserva pocos vuelos    (entre 0 y 1).
 
 - percentiles 25% y 75%: Los percentiles muestran que el 50% de los clientes reserva entre 0 y 8 vuelos, independientemente de su nivel educativo.

3. Prueba Estadística

Propuesta hipótesis:

H0: No hay una diferencia significativa en el número de vuelos reservados entre los grupos A y B.
H1: Hay una diferencia significativa en el número de vuelos reservados entre los grupos A y B

A = grupo sin estudios universitarios.
B = grupo con estudios universitarios.

Vamos a usar el Test Mann-Whitney U, prueba no paramétrica para comparar dos grupos cuando los datos no siguen una distribución normal.

In [38]:
#creamos función para agrupar por grupo con y sin estudios universitarios y creamos columna con esta info:
def agrupacion_grupos(education):
    
    if education in ["Bachelor", "Master", "Doctor"]:
        return "University Educated"
    else:
        return "Non-University Educated"

df_test["Group"] = df_test["Education"].apply(agrupacion_grupos)

In [40]:
df_test.sample(3)

,Flights Booked,Education,Group
252403,0,College,Non-University Educated
301385,16,College,Non-University Educated
229508,0,Bachelor,University Educated


In [49]:
#filtramos los datos de los grupos
#dividimos DataFrame en dos grupos para poder realizar el test:
grupo_a = df_test[df_test["Group"] == "Non-University Educated"]["Flights Booked"]
grupo_b = df_test[df_test["Group"] == "University Educated"]["Flights Booked"]

#visualizamos tamaño de ambos grupos
print(f"Número de registros en Grupo A, Non-University Educated: {len(grupo_a)}")
print('\n- - - -\n')
print(f"Número de registros en Grupo B, University Educated: {len(grupo_b)}")


Número de registros en Grupo A, Non-University Educated: 121175

- - - -

Número de registros en Grupo B, University Educated: 282585


Test Shapiro

In [52]:
#verificamos distribución normal
#usamos prueba shapiro:
p_value = shapiro(df_test["Flights Booked"]).pvalue
alpha = 0.05

if p_value > alpha:
    print(f"Los datos siguen una distribución normal (p-value = {p_value})")
else:
    print(f"Los datos no siguen una distribución normal (p-value = {p_value})")

Los datos no siguen una distribución normal (p-value = 0.0)


Test Mann-Whitney U

In [59]:
#no sigue distribución normal, usamos test de Mann-Whitney U
#statistic: es el valor U del Test Mann-Whitney, que indica la magnitud de la diferencia entre los dos grupos.
#p_value: valor p, se usa para determinar si hay diferencias significativas entre los grupos. Valor clave para la toma de decisiones.
statistic, p_value = mannwhitneyu(grupo_a, grupo_b, alternative='two-sided')

print(f'Test Mann-Whitney U: {statistic}')
print(f'Valor p: {p_value}')

alpha = 0.05
if p_value > alpha:
    print('No se rechaza la hipótesis.No hay diferencia significativa entre los grupos.')
else:
    print('Se rechaza la hipótesis nula. Hay diferencia significativa entre los grupos.')

Test Mann-Whitney U: 17216065553.0
Valor p: 0.002942081262120518
Se rechaza la hipótesis nula. Hay diferencia significativa entre los grupos.


Conclusiones:

- El nivel educativo influye significativamente en el número de vuelos reservados.

- Los clientes que tienen estudios universitarios tienden a revervar más vuelos.